# Semantic Segmentation with Unbihexium

[![CI](https://github.com/unbihexium-oss/unbihexium/workflows/CI/badge.svg)](https://github.com/unbihexium-oss/unbihexium/actions)
[![PyPI](https://img.shields.io/pypi/v/unbihexium.svg)](https://pypi.org/project/unbihexium/)
[![License](https://img.shields.io/badge/license-Apache--2.0-blue.svg)](LICENSE.txt)

**Author**: Unbihexium OSS Foundation  
**Version**: 1.0.0

---

## Purpose

Pixel-wise classification for:
- Land use / Land cover (LULC)
- Crop classification
- Water body detection
- Cloud masking

## Metrics

$$\text{IoU} = \frac{|A \cap B|}{|A \cup B|} = \frac{TP}{TP + FP + FN}$$

$$\text{Dice} = \frac{2|A \cap B|}{|A| + |B|}$$

| Metric | Formula | Range |
|--------|---------|-------|
| IoU | TP/(TP+FP+FN) | [0,1] |
| Dice | 2TP/(2TP+FP+FN) | [0,1] |
| Accuracy | (TP+TN)/N | [0,1] |

In [ ]:
import numpy as np
from pathlib import Path

def iou_score(pred: np.ndarray, target: np.ndarray) -> float:
    """Calculate Intersection over Union."""
    intersection = np.logical_and(pred, target).sum()
    union = np.logical_or(pred, target).sum()
    return intersection / (union + 1e-10)

def dice_score(pred: np.ndarray, target: np.ndarray) -> float:
    """Calculate Dice coefficient."""
    intersection = np.logical_and(pred, target).sum()
    return 2 * intersection / (pred.sum() + target.sum() + 1e-10)

## LULC Classes

In [ ]:
LULC_CLASSES = {
    0: "Background",
    1: "Water",
    2: "Forest",
    3: "Agriculture",
    4: "Urban",
    5: "Bare Soil",
    6: "Wetland",
    7: "Grassland"
}

for idx, name in LULC_CLASSES.items():
    print(f"Class {idx}: {name}")

## Run Segmentation Model

In [ ]:
try:
    import onnxruntime as ort
    
    model_path = Path("../model_zoo/assets/tiny/lulc_classifier_tiny/model.onnx")
    if model_path.exists():
        session = ort.InferenceSession(str(model_path))
        
        # Create synthetic input
        input_data = np.random.rand(1, 3, 32, 32).astype(np.float32)
        output = session.run(None, {"input": input_data})
        
        print(f"Output shape: {output[0].shape}")
        print(f"Classes predicted: {output[0].argmax(axis=1).max() + 1}")
    else:
        print("Model not found")
except Exception as e:
    print(f"Error: {e}")

## Evaluation

In [ ]:
# Synthetic evaluation
np.random.seed(42)
pred = np.random.rand(64, 64) > 0.5
target = np.random.rand(64, 64) > 0.4

print(f"IoU: {iou_score(pred, target):.3f}")
print(f"Dice: {dice_score(pred, target):.3f}")

---

**Copyright 2025 Unbihexium OSS Foundation. Apache-2.0 License.**